In [17]:
pip install nbformat

     ---------------------------------------- 0.0/61.2 kB ? eta -:--:--
     ------ --------------------------------- 10.2/61.2 kB ? eta -:--:--
     ------------------- ------------------ 30.7/61.2 kB 640.0 kB/s eta 0:00:01
     -------------------------------------- 61.2/61.2 kB 652.8 kB/s eta 0:00:00
   ---------------------------------------- 0.0/77.6 kB ? eta -:--:--
   ---------------------------------------- 77.6/77.6 kB 4.2 MB/s eta 0:00:00
   ---------------------------------------- 0.0/84.7 kB ? eta -:--:--
   -------------------------------------- - 81.9/84.7 kB 2.3 MB/s eta 0:00:01
   ---------------------------------------- 84.7/84.7 kB 1.6 MB/s eta 0:00:00
   ---------------------------------------- 0.0/188.7 kB ? eta -:--:--
   ----------------------- ---------------- 112.6/188.7 kB 3.3 MB/s eta 0:00:01
   ------------------------------------ --- 174.1/188.7 kB 2.1 MB/s eta 0:00:01
   ---------------------------------------- 188.7/188.7 kB 1.9 MB/s eta 0:00:00
Note: you 

data preprocess

In [2]:
import pandas as pd
import talib
import numpy as np
import vectorbt as vbt 
from vectorbt.portfolio.enums import OrderSide
from numba import njit

# 計算keltner channel
def keltner_channel(high, low, close, timeperiod = 20, nbdev = 2):
    middle_line = talib.EMA(close, timeperiod = timeperiod)
    atr = talib.ATR(high, low, close, timeperiod = timeperiod)
    
    # 上通道和下通道
    upper_channel = middle_line + nbdev * atr
    lower_channel = middle_line - nbdev * atr
    
    return upper_channel, middle_line, lower_channel

# 讀取data
data = pd.read_csv("3105.csv")
data = data.set_index('Date')
data.index = pd.to_datetime(data.index)

# 計算指標
upper_channel, middle_line, lower_channel = keltner_channel(data['High'], data['Low'], data['Close'])
upper, middle, lower = talib.BBANDS(data['Close'], timeperiod=20, nbdevup=2, nbdevdn=2, matype=0)
sar = talib.SAR(data['High'], data['Low'], acceleration=0.02, maximum=0.2)
five_day_min_low = talib.MIN(data['Low'], timeperiod=5) # 五天最低價當作long strategy的停損點
five_day_max_high = talib.MAX(data['High'], timeperiod=5) # 五天最高價當作short strategy的停損點

# 放入dataframe
data['SAR'] = sar
data['upper_channel'] = upper_channel
data['lower_channel'] = lower_channel
data['upper_bb'] = upper
data['lower_bb'] = lower
data['five_day_min_low'] = five_day_min_low
data['five_day_max_high'] = five_day_max_high

# 在squeeze中出現close突破upper bb的上漲k棒(long)
# 在squeeze中出現close跌破lower bb的下跌K棒(short)

# 最後改成只有squueeze
data['squeeze'] = (data['upper_bb'] <= data['upper_channel']) & (data['lower_bb']  >= data['lower_channel'])
data['breakthrough'] = (data['Close'] > data['Open']) & (data['Close'] >= data['upper_bb'])
data['breakdown'] = (data['Close'] < data['Open']) & (data['Close'] <= data['lower_bb'])
data['target_squeeze_long'] = data['squeeze'] 
data['target_squeeze_short'] = data['squeeze'] 


long and short strategy

long strategy 停利點改為下一個squeeze

In [7]:
# 台股手續費
def custom_fees(order):
    if order.side == OrderSide.Buy:
        return order.size * order.price * 0.001425  
    elif order.side == OrderSide.Sell:
        return order.size * order.price * 0.001425 + order.size * order.price * 0.003  
    return 0

# long strategy
signals_long = np.zeros(len(data))
condition1_long = False
partition_size_long = 0
stop_loss_long = 0
for i in range(3, len(data)):
    if data['target_squeeze_long'][i] and not condition1_long:
        condition1_long = True
    if partition_size_long == 0:
        if condition1_long and not data['squeeze'][i] and data['squeeze'][i-1] and (data['Close'][i] > data['SAR'][i]) and data['breakthrough'][i]:
            signals_long[i] = 1
            partition_size_long = 1
            stop_loss_long = data['five_day_min_low'][i]
        elif condition1_long and not data['squeeze'][i] and data['squeeze'][i-2] and (data['Close'][i] > data['SAR'][i]) and data['breakthrough'][i]:
            if data['Close'][i-1] > data['Open'][i-1] :
                signals_long[i] = 1
                partition_size_long = 1
                stop_loss_long = data['five_day_min_low'][i]
        elif condition1_long and not data['squeeze'][i] and data['squeeze'][i-3] and (data['Close'][i] > data['SAR'][i]) and data['breakthrough'][i]:
            if (data['Close'][i-1] > data['Open'][i-1]) &  (data['Close'][i-2] > data['Open'][i-2]):
                signals_long[i] = 1
                partition_size_long = 1
                stop_loss_long = data['five_day_min_low'][i]
        elif condition1_long and not data['squeeze'][i] and data['squeeze'][i-4] and (data['Close'][i] > data['SAR'][i]) and data['breakthrough'][i]:
            if (data['Close'][i-1] > data['Open'][i-1]) &  (data['Close'][i-2] > data['Open'][i-2]) & (data['Close'][i-3] > data['Open'][i-3]):   
                signals_long[i] = 1
                partition_size_long = 1
                stop_loss_long = data['five_day_min_low'][i]


    if partition_size_long == 1:
        # crossover = (data['Close'][i-1] > data['SAR'][i-1]) & (data['Close'][i] < data['SAR'][i])
        if data['squeeze'][i] or (data['Close'][i] <= stop_loss_long): # 停損跟停利
            signals_long[i] = -1
            condition1_long = False # condition1重置
            stop_loss_long = 0 # 停損點重置
            partition_size_long = 0 # 平倉
            
# short strategy
signals_short = np.zeros(len(data))
condition1_short = False
partition_size_short = 0
stop_loss_short = 0
for i in range(3, len(data)):
    if data['target_squeeze_short'][i] and not condition1_short:
        condition1_short = True
    if partition_size_short == 0:
        if condition1_short and not data['squeeze'][i] and data['squeeze'][i-1] and (data['Close'][i] < data['SAR'][i]) and data['breakdown'][i]:
            signals_short[i] = -1
            partition_size_short = -1
            stop_loss_short = data['five_day_max_high'][i]
        elif condition1_short and not data['squeeze'][i] and data['squeeze'][i-2] and (data['Close'][i] < data['SAR'][i]) and data['breakdown'][i]:
            if data['Close'][i-1] < data['Open'][i-1] :
                signals_short[i] = -1
                partition_size_short = -1
                stop_loss_short = data['five_day_max_high'][i]
        elif condition1_short and not data['squeeze'][i] and data['squeeze'][i-3] and (data['Close'][i] < data['SAR'][i]) and data['breakdown'][i]:
            if (data['Close'][i-1] < data['Open'][i-1]) &  (data['Close'][i-2] < data['Open'][i-2]):
                signals_short[i] = -1
                partition_size_short = -1
                stop_loss_short = data['five_day_max_high'][i]
        elif condition1_short and not data['squeeze'][i] and data['squeeze'][i-4] and (data['Close'][i] < data['SAR'][i]) and data['breakdown'][i]:
            if (data['Close'][i-1] < data['Open'][i-1]) &  (data['Close'][i-2] < data['Open'][i-2]) & (data['Close'][i-3] < data['Open'][i-3]): 
                signals_short[i] = -1
                partition_size_short = -1
                stop_loss_short = data['five_day_max_high'][i]


    if partition_size_short == -1:
        crossunder = (data['Close'][i-1] < data['SAR'][i-1]) & (data['Close'][i] > data['SAR'][i])
        if crossunder or (data['Close'][i] >= stop_loss_short):
            signals_short[i] = 1
            condition1_short = False # condition1重置
            stop_loss_short = 0 # 停損點重置
            partition_size_short = 0 # 平倉

entries_long = signals_long == 1
exits_long = signals_long == -1
entries_short = signals_short == -1
exits_short = signals_short == 1
pf_long = vbt.Portfolio.from_signals(data['Close'].values, 
                                     entries_long, 
                                     exits_long,
                                     entries_short,
                                     exits_short,
                                     fees = 0.001425, 
                                     freq='1D', 
                                     direction='both')
print(pf_long.stats().to_string()) # to_string()可以將全部結果攤開
'''entries_short = signals_short == -1
exits_short = signals_short == 1
pf_short = vbt.Portfolio.from_signals(data['Close'], entries_short, exits_short, direction='shortonly')'''
# print(pf_short.stats().to_string())


C:\Users\Admin\AppData\Local\Temp\ipykernel_12152\1326326238.py:15: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

C:\Users\Admin\AppData\Local\Temp\ipykernel_12152\1326326238.py:18: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

C:\Users\Admin\AppData\Local\Temp\ipykernel_12152\1326326238.py:22: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

C:\Users\Admin\AppData\Local\Temp\ipykernel_12152\1326326238.py:27: FutureWarning:

Series.__g

Start                                                  0
End                                                 1917
Period                                1918 days 00:00:00
Start Value                                        100.0
End Value                                     410.454889
Total Return [%]                              310.454889
Benchmark Return [%]                          208.240081
Max Gross Exposure [%]                             100.0
Total Fees Paid                                23.712491
Max Drawdown [%]                               26.357056
Max Drawdown Duration                  595 days 00:00:00
Total Trades                                          26
Total Closed Trades                                   25
Total Open Trades                                      1
Open Trade PnL                                 17.050472
Win Rate [%]                                        52.0
Best Trade [%]                                 55.679093
Worst Trade [%]                

C:\Users\Admin\AppData\Local\Temp\ipykernel_12152\1326326238.py:89: UserWarning:

direction has no effect if short_entries and short_exits are set



"entries_short = signals_short == -1\nexits_short = signals_short == 1\npf_short = vbt.Portfolio.from_signals(data['Close'], entries_short, exits_short, direction='shortonly')"

visualization

In [4]:
pf_long.plot().show()
# pf_short.plot().show()